# Entendiendo la Tokenización y el Embedding con Keras

A continuación, se presenta un ejemplo sencillo de tokenización y embedding utilizando la biblioteca Keras. El ejemplo consiste en una lista de cuatro frases que serán sometidas a los procesos de tokenización y embedding. Luego, estos resultados serán utilizados como entrada para una red neuronal.

In [ ]:
import numpy as np
from keras.preprocessing.text import Tokenizer
#from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

In [3]:
# Ejemplo de datos de entrada
texts = ['Esto es un ejemplo de texto para clasificar.',
         'Otro ejemplo de texto para probar el modelo.',
         'Un tercer ejemplo para completar el conjunto de datos.',
         'Último ejemplo para el ejemplo.']

labels = np.array([0, 1, 0, 1])  # Ejemplo de etiquetas

In [4]:
# Tokenización y preparación de datos
maxlen = 10  # Longitud máxima de secuencia
training_samples = 3  # Número de muestras de entrenamiento
validation_samples = len(texts) - training_samples  # Número de muestras de validación
max_words = 10000  # Número máximo de palabras en el vocabulario

Tokenizar es el proceso de **dividir un texto en partes más pequeñas** llamadas tokens. Estos tokens pueden ser **palabras individuales**, números, signos de puntuación o cualquier otra unidad significativa de texto.

En el procesamiento del lenguaje natural (NLP), la tokenización es una tarea fundamental que se realiza antes de procesar el texto en modelos de aprendizaje automático. La tokenización ayuda a convertir el texto en una forma que pueda ser utilizada **como entrada para el modelo.**

En resumen, tokenizar es el proceso de **dividir un texto** en partes más pequeñas (tokens) para su posterior procesamiento. Esto se hace generalmente para análisis de texto, como clasificación de texto, análisis de sentimientos, traducción automática, entre otros.

El argumento num_words en Tokenizer(num_words=max_words) especifica **el número máximo de palabras** que se deben tener en cuenta al tokenizar el texto.

Por ejemplo, si estableces num_words=10000, el tokenizador solo tendrá en cuenta **las 10,000 palabras más frecuentes** en el conjunto de datos y todas las demás palabras **se ignorarán**. Esto es útil para limitar el tamaño del vocabulario y, por lo tanto, el tamaño de los datos de entrada al modelo, lo que puede mejorar la eficiencia computacional y evitar problemas de sobreajuste.

In [5]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)

In [6]:
tokenizer.index_word

{1: 'ejemplo',
 2: 'para',
 3: 'de',
 4: 'el',
 5: 'un',
 6: 'texto',
 7: 'esto',
 8: 'es',
 9: 'clasificar',
 10: 'otro',
 11: 'probar',
 12: 'modelo',
 13: 'tercer',
 14: 'completar',
 15: 'conjunto',
 16: 'datos',
 17: 'último'}

In [7]:
word_index = tokenizer.word_index
print('Se encontraron %s tokens únicos.' % len(word_index))

Se encontraron 17 tokens únicos.


tokenizer.texts_to_sequences(texts) **convierte una lista de textos en una lista de secuencias de números enteros**. Cada número entero **representa la posición** de la palabra en el vocabulario generado por el tokenizador. Cada lista dentro de la lista de salida representa una secuencia de números enteros correspondientes a las palabras en el texto original, de acuerdo con el índice asignado por el tokenizador.

La codificación que se realiza con tokenizer.texts_to_sequences(texts) **no es una codificación one-hot**, sino que simplemente convierte cada palabra en una secuencia de números enteros de acuerdo con el índice de la palabra en el vocabulario.

In [8]:
sequences = tokenizer.texts_to_sequences(texts)
sequences

[[7, 8, 5, 1, 3, 6, 2, 9],
 [10, 1, 3, 6, 2, 11, 4, 12],
 [5, 13, 1, 2, 14, 4, 15, 3, 16],
 [17, 1, 2, 4, 1]]

pad_sequences(sequences, maxlen=maxlen) **ajusta la longitud** de todas las secuencias de números enteros **a una longitud máxima específica maxlen.**

Si una secuencia tiene más de maxlen elementos, **se truncará** para que tenga una longitud de maxlen. Si una secuencia tiene menos de maxlen elementos, **se rellenará con ceros** al principio de la secuencia para que tenga una longitud de maxlen.

In [9]:
data = pad_sequences(sequences, maxlen=maxlen, padding="pre")
data

array([[ 0,  0,  7,  8,  5,  1,  3,  6,  2,  9],
       [ 0,  0, 10,  1,  3,  6,  2, 11,  4, 12],
       [ 0,  5, 13,  1,  2, 14,  4, 15,  3, 16],
       [ 0,  0,  0,  0,  0, 17,  1,  2,  4,  1]])

## Capa de Embedding

La capa Embedding en Keras es una capa de procesamiento de texto que convierte números enteros positivos (índices de palabras) **en vectores de tamaño fijo**. **Cada palabra se representa por un vector** denso de valores reales (los parámetros del modelo son aprendidos automáticamente).

Aquí está el significado de los argumentos de Embedding(max_words, 8, input_length=maxlen):

- **input_dim:** El primer argumento **es el tamaño del vocabulario**, es decir, el número máximo **de palabras diferentes** que se pueden tener en cuenta. Las palabras fuera de este índice serán ignoradas.

- **output_dim:** El segundo argumento es la dimensión del espacio vectorial en el que se incrustarán las palabras. En este caso, **cada palabra** se representará como un vector **de longitud 8**. El embedding se aplica **a nivel de palabras, no a nivel de frases completas**. Cuando se aplica la capa de embedding en Keras, **cada palabra** en la secuencia se convierte en un vector de longitud fija de acuerdo con la dimensión del embedding especificada. Estos vectores son luego procesados por las capas posteriores de la red neuronal. Por lo tanto, después de aplicar la capa de embedding, cada palabra en la secuencia de entrada **se representa como un vector**. La representación de la secuencia completa se obtiene **combinando estos vectores**, generalmente **promediándolos o concatenándolos**, dependiendo de la arquitectura de la red neuronal.

- **input_length:** La longitud de las secuencias de entrada, que **debe ser la misma que la longitud de las secuencias después de aplicar pad_sequences**. En este caso, maxlen es la longitud máxima de las secuencias de entrada que se ajustó previamente utilizando pad_sequences. El argumento input_length en la capa Embedding **debe ser igual a la longitud de las secuencias después de aplicar el relleno (padding)**. Esto asegura que la capa Embedding **sepa qué longitud esperar para cada secuencia de entrada**. Entonces, si aplicaste pad_sequences con maxlen=maxlen, el argumento input_length de la capa Embedding debe ser igual a maxlen. Esto garantiza que las secuencias de entrada tengan la misma longitud que la que la capa Embedding espera.

In [10]:
# Definición del modelo
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=8, input_length=maxlen))

#Obtener la salida de la capa de embedding
embedded_data = model.predict(data)


1/1 [==============================] - 0s 97ms/step


Mostrar la forma de la salida

In [11]:
print("Forma de salida de la capa de embedding: ", embedded_data.shape)

Forma de salida de la capa de embedding:  (4, 10, 8)


La forma de la salida es (4, 10, 8), lo que significa que tenemos 4 secuencias en el lote, cada una con una longitud de 10 (debido al padding) y cada palabra representada por un vector de longitud 8 después

In [13]:
# Mostrar la salida de la capa de embedding
print("Salida de la capa de embedding:")
print(embedded_data[0])

Salida de la capa de embedding:
[[ 0.02984177 -0.02747539  0.03664869 -0.01651597 -0.03670455  0.04548267
   0.00334885 -0.01498439]
 [ 0.02984177 -0.02747539  0.03664869 -0.01651597 -0.03670455  0.04548267
   0.00334885 -0.01498439]
 [ 0.02323672 -0.04886472  0.02024076  0.03435335  0.00584662 -0.03813946
  -0.02302487  0.04820934]
 [-0.02668855  0.02641291  0.01014973  0.03130524  0.04248685  0.04340789
   0.04386213  0.02592975]
 [ 0.02296618  0.01268751 -0.00826931  0.02466289 -0.00603859  0.03556115
  -0.04260237  0.02507292]
 [-0.03043361 -0.0152789   0.01945398  0.00390643 -0.00664817 -0.04933877
   0.04003601  0.04925055]
 [ 0.0122527   0.00184907 -0.01474243 -0.00608959  0.03644953 -0.03419687
  -0.00430704 -0.0497017 ]
 [ 0.04405561  0.02223634 -0.04763883 -0.01063512 -0.02597965 -0.0151841
  -0.02315701 -0.03719201]
 [ 0.0383227   0.01189471 -0.00612334 -0.02129604 -0.0158491   0.04025948
   0.0486761   0.00324627]
 [ 0.00995102  0.01222322  0.01195465 -0.03428517 -0.0115840

Se continua con la construcción del modelo

In [14]:
#Se agregan más capas
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

In [15]:
# Entrenamiento del modelo
model.fit(data[:training_samples], labels[:training_samples],
          epochs=10,
          batch_size=32,
          validation_data=(data[training_samples:], labels[training_samples:]))

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 0.6949 - acc: 0.3333 - val_loss: 0.7025 - val_acc: 0.0000e+00
Epoch 2/10
1/1 [==============================] - 0s 54ms/step - loss: 0.6907 - acc: 0.3333 - val_loss: 0.7030 - val_acc: 0.0000e+00
Epoch 3/10
1/1 [==============================] - 0s 35ms/step - loss: 0.6865 - acc: 0.6667 - val_loss: 0.7035 - val_acc: 0.0000e+00
Epoch 4/10
1/1 [==============================] - 0s 50ms/step - loss: 0.6823 - acc: 1.0000 - val_loss: 0.7040 - val_acc: 0.0000e+00
Epoch 5/10
1/1 [==============================] - 0s 33ms/step - loss: 0.6782 - acc: 1.0000 - val_loss: 0.7045 - val_acc: 0.0000e+00
Epoch 6/10
1/1 [==============================] - 0s 36ms/step - loss: 0.6740 - acc: 1.0000 - val_loss: 0.7050 - val_acc: 0.0000e+00
Epoch 7/10
1/1 [==============================] - 0s 33ms/step - loss: 0.6699 - acc: 1.0000 - val_loss: 0.7054 - val_acc: 0.0000e+00
Epoch 8/10
1/1 [==============================] - 0s 35ms/step - loss: 

In [13]:
# Evaluación del modelo
loss, accuracy = model.evaluate(data[training_samples:], labels[training_samples:])
print('Accuracy:', accuracy)

1/1 [==============================] - 0s 35ms/step - loss: 0.6998 - acc: 0.0000e+00
Accuracy: 0.0
